In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 5000)

import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (15,11)
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
games = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
games

In [ ]:
players_original = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
players = players_original.copy()
players

TOC
* [Adding the plays data](#adding-plays)
* [Adding number of returns, kicks, and blocks](#adding-number-cols)
* [Part 1: Punters](#punters)
* * Punters dataset includes the number of punts, ratio of normal punts (as opposed to Aussie-style), ratios of how the ball lands, ratio of punts going the intended direction, average and maximum hang times
* [Part 2: Kickers](#kickers)
* * Kickers dataset includes number of kickoffs and field goals, mean and max kickoff distance, mean and max kickoff hangtime, ratio of kicks going the intended direction, ratio of touchbacks, ratio of kicks that go Out of Bounds, ratio of onside kicks (to normal kicks), recovery rate of those onside kicks, mean and max distance of succesful field goals, mean, minimum and max distance of missed field goals, as well as the number of kicks made and missed in the clutch - to take the lead or tie with under 3 minutes to play
* [Part 3: Returners](#returners)


<a id='top'></a>

# Augmenting the NFL players special teams dataset
In this notebook, I'm going to be using the data provided in the NFL big data bowl to add a ton of features to the special teams dataset. This is currently a work-in-progress but I'll upload versions of the dataset(s) to kaggle as I go.

The current version (first one) splits the players into 3 - punters, kickers, and returners - and simply has the number of kicks, punts, whatever-they-do as an extra column. I'm planning next on connecting it with the tracking and scouting data to get some more interesting fields.

For the moment, the dataset contains several self-evident fields:
* Player's NFL ID, a unique identifier;
* Height, Weight, Birthdate;
* Their college;
* Their (non special teams) position;
* Their name.

Right off the bat, we can get some information. For example, a look at the players by college:

In [ ]:
top25_colleges = players.groupby('collegeName').count().sort_values(by='nflId', ascending=False)['nflId'].head(25)

In [ ]:
# barchart of player colleges
top25_colleges.plot(kind='bar')

And by position:

In [ ]:
positions = players.groupby('Position').count().sort_values(by='nflId', ascending=False)['nflId']

# barchart of positions
positions.plot(kind='bar')

The purpose of this notebook will be to combine much of the other data supplied by the NFL into a new players dataset to facilitate this kind of analysis. 

### Bringing in plays data
[Return to top](#top)

<a id="adding-plays"></a>

The plays data will be the first thing I'm going to use to add the number of each type of play that each players was involved with. 

* First, the kickerId column needs to be adjusted so that I can differentiate between a punt, kickoff and field goal;
* Second, the returner column poses a slight problem for plays where there was a lateral involved. This will be handled in [Part 4: returners](id='returners');

In [ ]:
plays = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
plays

In [ ]:
plays.info()

Splitting kickoffs, punts and field goals - I'll make a new dataframe with only some relevant info and then adjust it so that instead of a single kickerId column, it'll be punter, kickoff kicker, or fg kicker. 

In [ ]:
kick_plays = plays.loc[plays['specialTeamsPlayType'].isin(['Kickoff', 'Punt', "Field Goal"])]
kick_plays

In [ ]:
# drop unecessary fields
kick_plays = kick_plays.drop(['playDescription',
                              'quarter','down',	'yardsToGo','possessionTeam',
                              'specialTeamsResult', 'yardlineSide','yardlineNumber',
                              'gameClock',	'penaltyCodes',	'penaltyJerseyNumbers',
                              'penaltyYards','preSnapHomeScore', 'preSnapVisitorScore','passResult',
                              'kickLength','kickReturnYardage', 'playResult','absoluteYardlineNumber'],
                             axis=1)

# add the new columns
kick_plays['ko_kickerId'] = np.nan
kick_plays['fg_kickerId'] = np.nan
kick_plays['punterId'] = np.nan

# fill the new columns 
for row in kick_plays.itertuples():
    if row.specialTeamsPlayType == "Kickoff":
        kick_plays.at[row.Index,'ko_kickerId'] = kick_plays.at[row.Index,'kickerId']
    elif row.specialTeamsPlayType == "Punt":
        kick_plays.at[row.Index,'punterId'] = kick_plays.at[row.Index,'kickerId']
    elif row.specialTeamsPlayType == "Field Goal":
        kick_plays.at[row.Index,'fg_kickerId'] = kick_plays.at[row.Index,'kickerId']
        
kick_plays

This is a function for adding in the number of each type of play. Note that under this version, it doesn't yet handle the returner column well where laterals are involved.

In [ ]:
def merge_play_counts(players_df, plays_df, col):
    """A function to add the number of kicks, kick blocks, and returns to a column
       Note: for now, it just drops the returners (by index) who pose the lateral problem
       Those players will be manually added back below """
    df = players_df.copy()
    
    # handling the returners
    # for now this drops the problematic ones, to be added back and dealt with
    if col == 'returnerId':
        plays_df = plays_df.drop([1753, 2394,5107,6688,11195,12503,16750,16928,16937,19367,19883])
        plays_df[col] = pd.to_numeric(plays_df[col])
        
    # get the number of whatever col I'm looking for into a temp df
    temp = pd.DataFrame(plays_df[col].value_counts())
    temp = temp.reset_index().rename(columns={col: f'num_{col[:-4]}s', 'index':'nflId'})
    
    # left join the temp df to the main one
    df = pd.merge(df, temp, how='left', on='nflId')
    
    # fill the nan's with 0's
    df[f'num_{col[:-4]}s'].fillna(value=0, inplace=True)
    
    return df
    

<a id="punters"></a>
## Part 1: Punters

[Return to top](#top)



In [ ]:
punters = players[players["Position"]=='P']
punters = merge_play_counts(punters, kick_plays, 'punterId')
punters.head()

In [ ]:
# plotting the punters that happen to be in those first 12, completely arbitrary
punters.head(12).plot.bar(x='displayName', y='num_punts')

### Scouting data
In the scouting data, first we need to get the plays for kickoffs and punts.

In [ ]:
scouting = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
scouting.head()

First, I need to get the plays referring to punts. From the [competition page](https://www.kaggle.com/c/nfl-big-data-bowl-2022/data), there are three values in the kickType field that refer to punts: N, normal punts; A, nose-down, Aussie style punts; R, Rugby-style punts.

In [ ]:
# getting number of each type of kick
scouting['kickType'].value_counts()

In [ ]:
# dataframe of only the punts
scouting_punts = scouting[scouting['kickType'].isin(['N','A','R'])]

# dropping some unnecessary fields
scouting_punts = scouting_punts.drop(['missedTackler','assistTackler','tackler',
                                      'kickoffReturnFormation','gunners','puntRushers',
                                      'specialTeamsSafeties','vises','returnDirectionIntended',
                                      'returnDirectionActual'], axis=1)

# the punt_map df is used to join the plays data (here only kick_plays) with the scouting data
punt_map = kick_plays[kick_plays['specialTeamsPlayType']=="Punt"].drop(['specialTeamsPlayType','kickerId','returnerId','kickBlockerId','ko_kickerId','fg_kickerId'], axis=1)
scouting_punts = scouting_punts.merge(punt_map, how='left',on=['gameId','playId'])

scouting_punts

Adding in some useful fields:

In [ ]:
# adding a column for punts that went the correct direction
scouting_punts['kickDirectionCorrect'] = scouting_punts['kickDirectionIntended']==scouting_punts['kickDirectionActual']

# removing the direction columns that are now useless
scouting_punts = scouting_punts.drop(['kickDirectionIntended','kickDirectionActual'], axis=1)

Check if the number of punts for each punter in the scouting data matches the number from the plays data:

In [ ]:
# counts records in the scouting data per punter
# renames the columns for matching
# merges with punter df 
punt_count_compare = pd.DataFrame(scouting_punts['punterId'].value_counts()).reset_index().rename(columns={'index':'nflId','punterId':'scouting_num_punts'}).merge(punters[['nflId','num_punts']], how='inner')

# adds a column to compare the numbers
punt_count_compare['difference'] = punt_count_compare['num_punts'] - punt_count_compare['scouting_num_punts']

punt_count_compare.sort_values('difference', ascending=False)

There are slight differences between the two datasets, however they are very small, so I'm going to leave it for the time being. First, I'll add up the categorical columns to get numbers for each punter.

Regarding how the punts were fielded, the following is taken directly from the [data page](#https://www.kaggle.com/c/nfl-big-data-bowl-2022/data):

kickContactType: Detail on how a punt was fielded, or what happened when it wasn't fielded (text).
Possible values:
* BB: Bounced Backwards
* BC: Bobbled Catch from Air
* BF: Bounced Forwards
* BOG: Bobbled on Ground
* CC: Clean Catch from Air
* CFFG: Clean Field From Ground
* DEZ: Direct to Endzone
* ICC: Incidental Coverage Team Contact
* KTB: Kick Team Knocked Back
* KTC: Kick Team Catch
* KTF: Kick Team Knocked Forward
* MBC: Muffed by Contact with Non-Designated Returner
* MBDR: Muffed by Designated Returner
* OOB: Directly Out Of Bounds

I'll seperate those into five: Caught, Ground, Kick Team Touched first, Out of Bounds, Out of Endzone

In [ ]:
# view the number of each of the categorical punts
punters_summed_dummy_columns = pd.get_dummies(scouting_punts, columns=['kickType','kickContactType','kickDirectionCorrect']).groupby(['punterId']).sum()

# styles of punting
punting_styles = punters_summed_dummy_columns[['kickType_A','kickType_N','kickType_R']]
punting_styles['normalPuntRatio'] = punting_styles['kickType_N']/(punting_styles['kickType_A'] + punting_styles['kickType_N'] + punting_styles['kickType_R'])
punting_styles = punting_styles.reset_index()
punting_styles['nflId'] = punting_styles['punterId'] 
punting_styles = punting_styles.drop(['kickType_A','kickType_N','kickType_R','punterId'], axis=1)


# punts fielded
punts_fielded = punters_summed_dummy_columns[['kickContactType_BB','kickContactType_BC','kickContactType_BF',
                                              'kickContactType_BOG','kickContactType_CC','kickContactType_CFFG',
                                              'kickContactType_DEZ','kickContactType_ICC','kickContactType_KTB',
                                              'kickContactType_KTC','kickContactType_KTF','kickContactType_MBC',
                                              'kickContactType_MBDR','kickContactType_OOB']]
punts_fielded_sum = punts_fielded['kickContactType_BB'] + punts_fielded['kickContactType_BC'] + punts_fielded['kickContactType_BF'] + punts_fielded['kickContactType_BOG'] + punts_fielded['kickContactType_CC'] + punts_fielded['kickContactType_CFFG'] + punts_fielded['kickContactType_DEZ'] + punts_fielded['kickContactType_ICC'] + punts_fielded['kickContactType_KTB'] + punts_fielded['kickContactType_KTC'] + punts_fielded['kickContactType_KTF'] + punts_fielded['kickContactType_MBC'] + punts_fielded['kickContactType_MBDR'] + punts_fielded['kickContactType_OOB']
punts_fielded['caughtRatio'] = (punts_fielded['kickContactType_BC'] + punts_fielded['kickContactType_CC'])/punts_fielded_sum
punts_fielded['groundRatio'] = (punts_fielded['kickContactType_BB'] + punts_fielded['kickContactType_BOG'] + punts_fielded['kickContactType_CFFG'])/punts_fielded_sum
punts_fielded['kickTeamTouchedFirstRatio'] = (punts_fielded['kickContactType_KTB'] + punts_fielded['kickContactType_KTF'] + punts_fielded['kickContactType_KTC'])/punts_fielded_sum
punts_fielded['outOfBoundsRatio'] = punts_fielded['kickContactType_OOB']/punts_fielded_sum
punts_fielded['outOfEZRatio'] = punts_fielded['kickContactType_DEZ']/punts_fielded_sum
punts_fielded = punts_fielded.reset_index()
punts_fielded['nflId'] = punts_fielded['punterId']
punts_fielded = punts_fielded.drop(['kickContactType_BB','kickContactType_BC','kickContactType_BF',
                                              'kickContactType_BOG','kickContactType_CC','kickContactType_CFFG',
                                              'kickContactType_DEZ','kickContactType_ICC','kickContactType_KTB',
                                              'kickContactType_KTC','kickContactType_KTF','kickContactType_MBC',
                                              'kickContactType_MBDR','kickContactType_OOB','punterId'], axis=1)


# kick direction intended
punts_kick_direction_intended = punters_summed_dummy_columns[['kickDirectionCorrect_True','kickDirectionCorrect_False']]
punts_kick_direction_intended['intendedDirectionRatio'] = punts_kick_direction_intended['kickDirectionCorrect_True']/(punts_kick_direction_intended['kickDirectionCorrect_True']+punts_kick_direction_intended['kickDirectionCorrect_False'])
punts_kick_direction_intended = punts_kick_direction_intended.reset_index()
punts_kick_direction_intended['nflId'] = punts_kick_direction_intended['punterId']
punts_kick_direction_intended = punts_kick_direction_intended.drop(['punterId','kickDirectionCorrect_True','kickDirectionCorrect_False'], axis=1)

In [ ]:
# merge those dataframes to the main punters dataset
punters = punters.merge(punting_styles, on='nflId')
punters = punters.merge(punts_fielded, on='nflId')
punters = punters.merge(punts_kick_direction_intended, on='nflId')

Next up, I want to get the mean and max hangtimes for each punter (hangtime refers to how long the ball stays in the air).

In [ ]:
# drop unecessary columns
punt_hts = scouting_punts.drop(['gameId','playId','snapDetail','snapTime','operationTime','kickType','kickContactType','kickDirectionCorrect'], axis=1)

# calculate mean and max ht by punter
punt_hts = punt_hts.groupby('punterId').agg(meanHangTime = ('hangTime','mean'),maxHangTime = ('hangTime','max')).reset_index()
punt_hts = punt_hts.rename(columns={'punterId':'nflId'})

# join it to the main set
punters = punters.merge(punt_hts, on='nflId')

In [ ]:
punters.sort_values('meanHangTime', ascending=False).head()

In [ ]:
def get_top_five(df, column):
    return df.sort_values(column, ascending=False)[['displayName', 'num_punts',column]].head()

Brett Kern put 15% of his punts out of bounds!

In [ ]:
get_top_five(punters, 'outOfBoundsRatio')

Feel free to take this in csv if you'd like:

In [ ]:
punters.to_csv('punters.csv')

<a id='kickers'></a>
## Part 2: Kickers

[Return to Top](#top)

In [ ]:
# isolate kickers
kickers = players[players["Position"]=="K"]

# get the proper id's
kickers = merge_play_counts(kickers, kick_plays, 'ko_kickerId')
kickers = merge_play_counts(kickers, kick_plays, 'fg_kickerId')

kickers

In [ ]:
kickers.head(12).plot.bar(x='displayName', y='num_ko_kicks')

<a id='kickoffs'></a>
### Part 2a, Kickoffs
What I want to look at:
* Deep kick average distance
* Touchback ratio
* OOB ratio
* Onside ratio
* Intended vs Actual Directions

In [ ]:
# get kickoffs from the plays df
kickoffs = plays[plays['specialTeamsPlayType'] == 'Kickoff']

# drop unecessary columns
kickoffs = kickoffs.drop(['playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'specialTeamsPlayType',
       'kickBlockerId', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers',
       'penaltyYards', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult',],axis=1)

kickoffs.head()

In [ ]:
# getting number of each type of kick
scouting['kickType'].value_counts()

In the scouting data, possible values for kickoff plays:

* D: Deep - your normal deep kick with decent hang time
* F: Flat - different than a Squib in that it will have some hang time and no roll but has a lower trajectory and hang time than a Deep kick off
* K: Free Kick - Kick after a safety
* O: Obvious Onside - score and situation dictates the need to regain possession. Also the hands team is on for the returning team
* P: Pooch kick - high for hangtime but not a lot of distance - usually targeting an upman
* Q: Squib - low-line drive kick that bounces or rolls considerably, with virtually no hang time
* S: Surprise Onside - accounting for score and situation an onsides kick that the returning team doesn’t expect. Hands teams probably aren't on the field
* B: Deep Direct OOB - Kickoff that is aimed deep (regular kickoff) that goes OOB directly (doesn't bounce)


In [ ]:
# dataframe of only the kickoffs
scouting_kos = scouting[scouting['kickType'].isin(['D','F','K','O','P','Q','S','B'])]

scouting_kos['kickDirectionCorrect'] = scouting_kos['kickDirectionIntended']==scouting_kos['kickDirectionActual']

# drop unecessary columns
scouting_kos = scouting_kos.drop(['snapDetail','snapTime','operationTime',
                                  'kickDirectionIntended','kickDirectionActual',
                                  'missedTackler','assistTackler','tackler',
                                  'gunners','puntRushers','vises',
                                  'specialTeamsSafeties','kickContactType',
                                  'kickoffReturnFormation'], axis=1)

# join with the plays df
kos = scouting_kos.merge(kickoffs, on=['gameId','playId'])

kos.head()

Getting deep kickoff average distance:

In [ ]:
# calculating the mean and max kickoff distances and hangtimes on deep kicks
deep_kos = kos[kos['kickType']=='D']
deep_kos = deep_kos.groupby('kickerId').agg(meanKickoffDistance=('kickLength','mean'),
                                            maxKickoffDistance=('kickLength','max'),
                                            meanKickoffHangtime=('hangTime','mean'),
                                            maxKickoffHantime=('hangTime','max')).reset_index().rename(columns={'kickerId':'nflId'})
# join it to the main set
kickers = kickers.merge(deep_kos, on='nflId', how='left')

In [ ]:
kos[kos['kickType']=='D']['specialTeamsResult'].value_counts()

In [ ]:
# view the number of each of the categorical punts
# it's going to sum up the gameId's, playId's, hangtimes, etc over each player as well, which is obviously nonsensical, but I'm not using these so it doesn't matter in this context
# those will get dropped from the df before it is merged with the kickers df
kickers_summed_dummy_columns = pd.get_dummies(kos, columns=['kickType','specialTeamsResult','kickDirectionCorrect']).groupby(['kickerId']).sum()

# column for the number of kicks
kickers_summed_dummy_columns['numKickoffs'] = kickers_summed_dummy_columns['kickType_B'] + kickers_summed_dummy_columns['kickType_D'] + kickers_summed_dummy_columns['kickType_F'] + kickers_summed_dummy_columns['kickType_K'] + kickers_summed_dummy_columns['kickType_O'] + kickers_summed_dummy_columns['kickType_P'] + kickers_summed_dummy_columns['kickType_Q'] + kickers_summed_dummy_columns['kickType_S'] 

# direction intended
kickers_summed_dummy_columns['intendedDirectionRatio'] = kickers_summed_dummy_columns['kickDirectionCorrect_True']/(kickers_summed_dummy_columns['kickDirectionCorrect_True']+kickers_summed_dummy_columns['kickDirectionCorrect_False'])

# calculate touchback ratio on deep kicks 
kickers_summed_dummy_columns['touchbackRatio'] = kickers_summed_dummy_columns['specialTeamsResult_Touchback']/kickers_summed_dummy_columns['kickType_D']

# calculate out of bounds ratio
kickers_summed_dummy_columns['OOBRatio'] = kickers_summed_dummy_columns['kickType_B']/kickers_summed_dummy_columns['numKickoffs']

# calculate onside ratio and recovery rate
kickers_summed_dummy_columns['onsideRatio'] = kickers_summed_dummy_columns['kickType_O']/kickers_summed_dummy_columns['numKickoffs']
kickers_summed_dummy_columns['onsideRecoveryRate'] = kickers_summed_dummy_columns['specialTeamsResult_Kickoff Team Recovery']/kickers_summed_dummy_columns['kickType_O']

# fix the df for joining
kickers_summed_dummy_columns = kickers_summed_dummy_columns.reset_index().rename(columns={'kickerId':'nflId'})
kickers_summed_dummy_columns = kickers_summed_dummy_columns.drop(kickers_summed_dummy_columns.columns[[x for x in range(1,26)]], axis=1)

# join the new stats columns to the main kickers df
kickers = kickers.merge(kickers_summed_dummy_columns, on='nflId')

In [ ]:
kickers.head()

<a id='field-goals'></a>
### Part 2b, Field Goals

Fields to add:
* Accuracy
* Distance
* Clutch kicks made (last two minutes of a game to win or tie)

In [ ]:
games.head()

In [ ]:
def encodeKickingTeamScore(df):
    if df['possessionTeam'] == df['homeTeamAbbr']:
        return df['preSnapHomeScore']
    else:
        return df['preSnapVisitorScore']
    
def encodeDefendingTeamScore(df):
    if df['possessionTeam'] == df['visitorTeamAbbr']:
        return df['preSnapHomeScore']
    else:
        return df['preSnapVisitorScore']

In [ ]:
def clutchKickIndicator(df):
    """Function to apply to the df to indicate that a given kick was made in the clutch - last three minutes of the game to tie or take the lead"""
    if df['clutchKickIndicator']:
        if 0<= df['defendingTeamScore']-df['kickingTeamScore'] <= 3:
            if df['specialTeamsResult']=='Kick Attempt Good':
                return 1
            elif df['specialTeamsResult']=='Kick Attempt No Good':
                return 0
    else:
        return np.NaN

In [ ]:
field_goals = plays[plays['specialTeamsPlayType'] == 'Field Goal'].drop(['playDescription', 
                                                                         'specialTeamsPlayType', 'yardlineSide','yardlineNumber',
                                                                         'returnerId', 'kickBlockerId', 'yardsToGo',
                                                                         'penaltyCodes', 'penaltyJerseyNumbers',
                                                                         'penaltyYards', 'passResult',
                                                                         'kickReturnYardage', 'playResult',
                                                                         ], axis=1)

# bring in the home and away columns from the games dataset 
field_goals = field_goals.merge(games[['gameId','homeTeamAbbr','visitorTeamAbbr']], on='gameId')

# add the clutch kick indicator
field_goals['clutchKickIndicator'] = (field_goals['gameClock'].str.slice(start=0, stop=2).isin(['02','01','00'])) & (field_goals['quarter'] == 4)

# add the more useful score columns
field_goals['kickingTeamScore'] = field_goals.apply(encodeKickingTeamScore, axis=1).astype('int64')
field_goals['defendingTeamScore'] = field_goals.apply(encodeDefendingTeamScore, axis=1).astype('int64')

# add the clutch kick made and missed for stats
field_goals['clutchKickMade'] = field_goals.apply(clutchKickIndicator, axis=1)
field_goals['clutchKickMissed'] = field_goals['clutchKickMade'] == 0

# get rid of a couple columns now made redundant
field_goals = field_goals.drop(['preSnapHomeScore','preSnapVisitorScore',], axis=1)

# mean and max kicks made
kicks_made_stats = field_goals[field_goals['specialTeamsResult'] == 'Kick Attempt Good'].groupby('kickerId').agg(meanKickLength_hit=('kickLength','mean'),
                                                                                                                 maxKickLength_hit=('kickLength','max'),
                                                                                                                 clutchKicksMade=('clutchKickMade','sum')).reset_index().rename(columns={'kickerId':'nflId'})

# mean, min and max kicks missed
kicks_missed_stats = field_goals[field_goals['specialTeamsResult'] == 'Kick Attempt No Good'].groupby('kickerId').agg(meanKickLength_missed=('kickLength','mean'),
                                                                                                                 maxKickLength_missed=('kickLength','max'),
                                                                                                                 shortestKickLength_missed=('kickLength','min'),
                                                                                                                 clutchKicksMissed=('clutchKickMissed','sum')).reset_index().rename(columns={'kickerId':'nflId'})

# finally, add these to the kickers df
kickers = kickers.merge(kicks_made_stats, on='nflId')
kickers = kickers.merge(kicks_missed_stats, on='nflId')

In [ ]:
# longest kicks?
kickers.sort_values('maxKickLength_hit', ascending=False).head(12)

Feel free to take this in csv if you'd like:

In [ ]:
kickers.to_csv('kickers.csv')

<a id='returners'></a>
## Part 3: Returners

[Return to Top](#top)

In [ ]:
returners = merge_play_counts(players, kick_plays, 'returnerId')
returners = returners[returners['num_returns'] > 0]
returners

11 kicks and punts involved laterals where multiple players could be considered a returner. 

In [ ]:
returns_with_laterals = plays[plays['returnerId'].str.contains(';', na=False)][['playDescription','returnerId']]
returns_with_laterals

[Return to Top](#top)


In [ ]:
returners.sort_values(by='num_returns', ascending=False)

Now I'm going to add in an extra return for every player who touched the ball in a return involving a lateral. There were also two players (Terry Godwin, Chris Claybrooks) who were on two kickoffs involving laterals, so they'll get two extra.



In [ ]:
returns_with_laterals

In [ ]:
# nflId's of the players involved with the above lateral plays
ids = [41238,43880,43329,44029,46130,36473,46221,44923,45555,46309,42718,46277,42094,52631,48020,52737,45719,46506,46669]

# for the two guys with multiple such plays
other_ids = [52631, 48020]

In [ ]:
returners.loc[returners['nflId'].isin(ids), 'num_returns'] += 1
returners.loc[returners['nflId'].isin(other_ids), 'num_returns'] += 1

In [ ]:
returners.sort_values(by=['num_returns'], ascending=False)

[Return to Top](#top)

In [ ]:
returners.head(12).plot.bar(x='displayName', y='num_returns')

## *TO BE CONTINUED....*